In [1]:
import networkx as nx
import sys
import os
from termcolor import colored
import json
from tqdm import tqdm
import matplotlib.pyplot as plt
from matplotlib.pyplot import figure
from itertools import count
import math
from IPython.display import clear_output

In [2]:
CLASSES_PATH = os.path.dirname(os.path.abspath('../../Classes/'))
if not (CLASSES_PATH in sys.path):
    sys.path.append(CLASSES_PATH)
from Classes.Files_Handler_Class import Files_Handler
from Classes.Resd_Network_Infos_Class import Resd_Network_Infos
from Classes.SIR_Diffusion_Model_Class import SIR_Diffusion_Model
from Classes.Get_Past_Results_Class import Get_Past_Results
from Classes.Network_Infos_Writer_Class import Network_Infos_Writer
from Classes.Bcolors_Class import Bcolors as bcolors


In [3]:
seed_size_list = [# [1, 4, 8, 12, 16, 20, 24, 28],
                  # [1, 4, 8, 12, 16, 20, 24, 28]
                  # [1, 3, 6, 9, 12, 15, 17, 19],
                  [15, 17, 19]# [1, 3, 6, 9, 12, 15, 17, 19]
                  # [1, 3, 6, 9, 12, 15, 17, 19]
                  # [1, 3, 6, 9, 12, 15, 17, 19]
                  # [1, 2, 4, 6, 8, 10, 12, 14],
                  # [1, 2, 3, 4, 5, 6, 8, 10],
                  # [1, 2, 3, 4, 5],
                  # [1, 2, 3, 4, 5]
                  ]
datasets_list = [# '01 - Sanremo2016_final (55897 Node and 3 Layer)/Sanremo2016_final.edgeslist',
                 # '02 - C_H_N (45583 Node and 3 Layer)/C_H_N.edgeslist'
                 # '03 - higgs-reply_network (38918 Node and 20 Layer)/higgs-reply_network.edgeslist'
                 '04 - 2 AS (33752 Node and 7 Layer)/2 AS.edgeslist'
                 # '05 - friendfeed_ita (21006 Node and 3 Layer)/friendfeed_ita.edgeslist',
                 # '06 - Figs S1-S5 (10000 Node and 5 Layer)/Figs S1-S5.edgeslist'
                 # '07 - fftwyt (6407 Node and 3 Layer)/fftwyt.edgeslist',
                 # '08 - Irvine (1663 Node and 51 Layer)/Irvine.edgeslist',
                 # '09 - Wainwright (217 Node and 36 Layer)/Wainwright.edges',
                 # '10 - realitycommons (84 Node and 5 Layer)/Relationshipsfromsurveys.edgelist'
                
                ]

root_file_path = 'D:/Masters thesis/Methods For Compare/Datasets/' 


color_list = ["light_red", "light_green", "light_yellow",
                    "light_blue","light_magenta", "light_cyan",
                    "blue", "red", "white", "green", "yellow",
                        "magenta", "cyan", ]
tqdm_color_list = ['blue', 'red', 'green', 'cyan', 'magenta', 'yellow', 'black', 'white']


files_handler_obj = Files_Handler()
SIR_diffusion_model_obj = SIR_Diffusion_Model()
resd_network_infos_obj = Resd_Network_Infos()


source_code_path = str(os.getcwd())
source_code_path = source_code_path.replace("\\", "/")

draw_status = False

In [4]:
def draw_networ(graphs_of_network):
    # get unique groups
    for j, graph in enumerate(graphs_of_network):
        if len(graph) > 0:
            groups = set([None])
            mapping = dict(zip(sorted(groups),count()))
            nodes = graph.nodes()
            colors = [mapping[None] for n in nodes]

            # drawing nodes and edges separately so we can capture collection for colobar
            pos = nx.spring_layout(graph)
            ec = nx.draw_networkx_edges(graph, pos, alpha=0.3)
            labels = nx.draw_networkx_labels(graph, pos=pos)
            nc = nx.draw_networkx_nodes(graph, pos, label=labels, nodelist=nodes, node_color=colors, cmap=plt.cm.jet)
            plt.colorbar(nc)
            plt.axis('off')
            plt.show()

def get_tuple_list_len(inp_tuple:tuple):
    return len(inp_tuple[1])



In [5]:
for i, path in enumerate(datasets_list):
    file_path = root_file_path + path
    for item in seed_size_list[i]:
        print(path, item)
        seed_set_size = item
#--------------------------------------------------------------------------------------------------------------------------
        if file_path is None or file_path == '':
            sys.exit("File Selection Canceled !")
        file_info = files_handler_obj.get_file_path_info(file_path)
        network_name = file_info['name']
        network_type = file_info['type']
        network_path = file_info['path']
        if network_name == "":
            sys.exit("Dont Network Selection!")
#--------------------------------------------------------------------------------------------------------------------------
        get_past_results_obj = Get_Past_Results(file_info['path'], file_info['name'])
        network_infos_writer_obj = Network_Infos_Writer(file_info['path'], file_info['name'])
#--------------------------------------------------------------------------------------------------------------------------
        (
            network_layers_info,
            network_layers_nodes,
            entra_layer_edges,
            entra_layer_edges_features,
            inter_layer_edge,
        ) = resd_network_infos_obj.read_nodeFrom_layerFrom_nodeTo_layerTo(
            network_path, network_name, network_type
        )
#--------------------------------------------------------------------------------------------------------------------------
        print(file_info['name'], '\n')
        sum_of_layers_nodes = 0
        network_layers_count = len(network_layers_info)
        graphs_of_network = [None] * network_layers_count
        network_entier_edges = ""
        layers_nodes_infect_scale = []

        i = 0
        j = 0
        while i < network_layers_count:
            graphs_of_network[i] = nx.Graph()
            network_layers_nodes[i] = list(set(network_layers_nodes[i]))
            layers_nodes_infect_scale.append({})
            if len(network_layers_nodes[i]) > 0:
                # graphs_of_network[i].add_nodes_from(network_layers_nodes[i])
                graphs_of_network[i].add_edges_from(entra_layer_edges[i])

                graphs_of_network[i].graph["id"] = i
                graphs_of_network[i].graph["node_num"] = graphs_of_network[i].number_of_nodes()
                graphs_of_network[i].graph["edge_num"] = graphs_of_network[i].number_of_edges()

                print(colored("Layer "  + str(i) + ": " + str(graphs_of_network[i].number_of_nodes()) + " Node And " +
                            str(graphs_of_network[i].number_of_edges()) + " Edge", color_list[j]))
                # print(colored(graphs_of_network[i].graph['k_shell_info'], color_list[i]))
            i += 1
            j += 1
            if j >= len(color_list):
                j = 0

        network_entier_nodes_list = []
        for item in network_layers_nodes:
            network_entier_nodes_list += item
            sum_of_layers_nodes += len(item)

        network_entier_nodes_list = list(set(network_entier_nodes_list))
        network_entier_nodes_count = len(network_entier_nodes_list)
        print()
        print("network entier nodes : " + colored(str(network_entier_nodes_count), "yellow"))
        CIM_path = files_handler_obj.make_dir(file_info['path'], 'CIM')
#--------------------------------------------------------------------------------------------------------------------------
        if draw_status:
            draw_networ(graphs_of_network)
        cliques, cliques_load_flag = get_past_results_obj.load_results_from_file(CIM_path, 'cliques')
#--------------------------------------------------------------------------------------------------------------------------
        if cliques_load_flag == False:
            cliques = []
            for j, graph in enumerate(graphs_of_network):
                cliques.append([])
                if graph.number_of_edges() > 0:
                    print(bcolors.HEADER + f'Layer {graph.graph["id"]}: {graph}' + bcolors.ENDC)
                    temp_cliques = list(nx.find_cliques(graph)) # The number of maximal cliques in graph
                    # print(j, len(temp_cliques))

                    def clique_nodes_max_degee(clique):
                        return max(list(graph.degree(clique)), key=lambda item: item[1])[1]
                    cliques[j] = sorted(temp_cliques, key=clique_nodes_max_degee, reverse=True)
                    # print(j, len(temp_cliques))

                    cliques[j] = sorted(cliques[j], key=len, reverse=True)
                    # print(j, len(temp_cliques))

                    i = 0
                    cliques_counter = len(cliques[j])
                    pbar = tqdm(total=cliques_counter)
                    pbar.colour = 'blue'
                    pbar.unit = ' clique'
                    p_itr = 1
                    while i in range(cliques_counter):
                        c = i + 1
                        while c in range(cliques_counter):
                            if c > (len(cliques[j]) - 1):
                                break
                            subscribe_flag = False
                            subscribe_flag_counter = 0
                            for item in cliques[j][c]:
                                if item in cliques[j][i]:
                                    subscribe_flag = True
                                    break
                            if subscribe_flag == True:
                                subscribe_flag_counter += 1
                                if len(cliques[j][i]) == cliques[j][c]:
                                    clique_i_max_degee = max(list(graph.degree(cliques[j][i])), key=lambda item: item[1])[1]
                                    clique_c_max_degee = max(list(graph.degree(cliques[j][c])), key=lambda item: item[1])[1]
                                    if clique_i_max_degee >= clique_c_max_degee:
                                        del cliques[j][c]
                                    else:
                                        del cliques[j][i]
                                else:
                                    del cliques[j][c]
                            else:
                                c += 1
                        if i > (cliques_counter - subscribe_flag_counter):
                            break
                        else:
                            i += 1
                        pbar.set_description(f'clique {p_itr}')
                        p_itr += 1
                        pbar.update(1)
                    pbar.close()
                
                    # print(j, len(temp_cliques), '\n')
#--------------------------------------------------------------------------------------------------------------------------
        if cliques_load_flag == False:
            temp_cliques = []
            pbar = tqdm(total=len(cliques))
            pbar.colour = 'red'
            pbar.unit = ' clique'
            p_itr = 1
            for j, jtem in enumerate(cliques):
                for i, item in enumerate(jtem):
                    temp_cliques.append((j, cliques[j][i]))  
                pbar.set_description(f'clique {p_itr}')
                p_itr += 1
                pbar.update(1)
            pbar.close()
#--------------------------------------------------------------------------------------------------------------------------
        if cliques_load_flag == False:
            # cliques = [elem for twod in cliques for elem in twod]
            cliques = sorted(temp_cliques, key=get_tuple_list_len, reverse=True)
            del temp_cliques
#--------------------------------------------------------------------------------------------------------------------------
        if cliques_load_flag == False:
            network_infos_writer_obj.write_results_in_file(CIM_path, 'cliques', cliques)
#--------------------------------------------------------------------------------------------------------------------------
        seed_set = []
        nomc = len(cliques)
        min_stock = seed_set_size / nomc
        upto = seed_set_size % nomc
        soc = []
        case3_flag = False
        for item in cliques:
            clique_len = len(item[1])
            soc.append(clique_len)
            if clique_len < min_stock:
                case3_flag = True

        print(min_stock, upto)
#--------------------------------------------------------------------------------------------------------------------------
        if nomc >= seed_set_size:
            i = 0
            while i < len(cliques):
                clique_max_degree_node = max(list(graphs_of_network[cliques[i][0]].degree(cliques[i][1])), key=lambda item: item[1])
                if len(seed_set) <= 0:
                    seed_set.append(clique_max_degree_node[0])
                elif not(clique_max_degree_node[0] in seed_set):
                    seed_set.append(clique_max_degree_node[0])
                if(len(seed_set) == seed_set_size):
                    break
                i += 1
        elif nomc <= seed_set_size:
            i = 0
            while i < len(cliques):
                j = 0
                while j < math.floor(min_stock):
                    clique_max_degree_node = max(list(graphs_of_network[cliques[i][0]].degree(cliques[i][1])), key=lambda item: item[1])
                    cliques[i][1].remove(clique_max_degree_node[0])
                    if len(seed_set) <= 0:
                        seed_set.append(clique_max_degree_node[0])
                        j += 1
                    elif not(clique_max_degree_node[0] in seed_set):
                        j += 1
                        seed_set.append(clique_max_degree_node[0])
                    if(len(seed_set) == seed_set_size) or (len(cliques[i][1]) <= 0):
                        break
                if len(seed_set) == seed_set_size:
                    break
                i += 1
            i = 0
            while i < upto:
                clique_max_degree_node = max(list(graphs_of_network[cliques[i][0]].degree(cliques[i][1])), key=lambda item: item[1])
                if len(seed_set) <= 0:
                    seed_set.append(clique_max_degree_node[0])
                elif not(clique_max_degree_node[0] in seed_set):
                    seed_set.append(clique_max_degree_node[0])
                if(len(seed_set) == seed_set_size):
                    break
                i += 1
            
#--------------------------------------------------------------------------------------------------------------------------
        print('seed_set_size" ', seed_set_size)
        print('seed_set: ', seed_set)
#--------------------------------------------------------------------------------------------------------------------------
        print(f"Network name: {network_name}")
        print(f"Seed set size: {seed_set_size}")
        beta = 0.01
        landa = 0.7
        epoch = 10000

        infection = SIR_diffusion_model_obj.synchronous_SIR_multilayer_with_seed_set_model(graphs_of_network, seed_set, beta, landa, epoch, network_entier_nodes_list)

        sir_results_infos = {}
        sir_results_infos['infection'] = infection
        sir_results_infos['percentage'] = infection / network_entier_nodes_count
        sir_results_infos['seed_set'] = seed_set

        network_infos_writer_obj.write_results_in_file(CIM_path, f'infection k={seed_set_size} beta={beta} landa{landa} epoch{epoch}', sir_results_infos)
        print(f"Network entier nodes count: {network_entier_nodes_count}")
        print(f"Infected nodes count: {infection}")
        print(f"Percentage of infection: {sir_results_infos['percentage']}")

#--------------------------------------------------------------------------------------------------------------------------
        clear_output(wait=False)
#--------------------------------------------------------------------------------------------------------------------------
print('Finished!')

Finished!


In [6]:
import os
# os.system('shutdown -s')